In [4]:
import torch
import clip
from PIL import Image
import numpy as np
from tqdm import tqdm

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("rn50/test.pt", device=device)

In [3]:
import os
try:
    os.mkdir("rn50")
except:
    pass
torch.save(model.state_dict(),"rn50/rn50.pt")

In [8]:
import json
# 从txt文件中读取类名
classes_file = "./data/classes.txt"
with open(classes_file, "r") as f:
    classes = [line.strip() for line in f.readlines()]
files=json.load(open("data/imnames_A.json"))

batch_size = 64

In [6]:
# datasets = []

# for i in tqdm(range(0, len(files), batch_size)):
#     batch_files = files[i:i + batch_size]
#     batch_images = [preprocess(Image.open("data/dataset_A/" + file.split("/")[-1])) for file in batch_files]
#     batch_tensors = torch.stack(batch_images)
#     datasets.append(batch_tensors)

100%|██████████| 344/344 [03:38<00:00,  1.58it/s]


In [10]:
text = clip.tokenize(classes).to(device)

In [9]:
import pickle
# pickle.dump(datasets,open("datasets.pkl","wb"))
datasets=pickle.load(open("datasets.pkl","rb"))

In [11]:
impreds=[]
with torch.no_grad():
    for imgs in tqdm(datasets):
        imgs = imgs.to(device)
        
        # image_features = model.encode_image(imgs)
        # text_features = model.encode_text(text)
        logits_per_image, logits_per_text = model(imgs, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
        impreds.extend(probs.tolist())

impreds_ = np.array(impreds)

  0%|          | 0/344 [00:00<?, ?it/s]/data/wangjieyang/anaconda3/envs/vit/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 344/344 [01:04<00:00,  5.32it/s]


In [12]:
name_pred=[]
for i,pred in enumerate(impreds_):
    # print(pred)
    name_pred.append(classes[pred.argmax()])
json.dump(name_pred,open("name_preds.json","w"))

In [13]:
json.dump(impreds,open("impreds.json","w"))